In [54]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
#from libs.utils import *
import matplotlib.pyplot as plt


In [55]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [56]:
x_tensor = tf.reshape(x, [-1, 28, 28, 1])



In [57]:
filter_size = 5
n_filters = 16
filter_shape = [filter_size, filter_size, 1, n_filters]
W1 = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[n_filters]))
conv1 = tf.nn.conv2d(input=x_tensor,
                 filter=W1,
                 strides=[1, 2, 2, 1],
                 padding='SAME')
h1=tf.nn.bias_add(conv1, b1)
z1 = tf.nn.relu(h1)



filter_size = 5
n_filters = 16
filter_shape = [filter_size, filter_size, n_filters, n_filters]
W2 = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[n_filters]))
conv2 = tf.nn.conv2d(input=conv1,
                 filter=W2,
                 strides=[1, 2, 2, 1],
                 padding='SAME')
h2=tf.nn.bias_add(conv2, b2)
z2 = tf.nn.relu(h2)



conv_flat = tf.reshape(h2, [-1, 7 * 7 * n_filters])
n_fc1 = 1024
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * n_filters, n_fc1], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[n_fc1]))
h_fc1 = tf.matmul(conv_flat, W_fc1) + b_fc1
z_fc1 = tf.nn.relu(h_fc1)



W_fc2 = tf.Variable(tf.truncated_normal([n_fc1,10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
y_pred = tf.nn.softmax(tf.matmul(z_fc1, W_fc2) + b_fc2)

In [58]:
cross_entropy = -tf.reduce_sum(y * tf.log(y_pred))
optimizer = tf.train.AdamOptimizer().minimize(cross_entropy)

# %% Monitor accuracy
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

# %% We now create a new session to actually perform the initialization the
# variables:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# %% We'll train in minibatches and report accuracy:
batch_size = 100
n_epochs = 5
for epoch_i in range(n_epochs):
    for batch_i in range(mnist.train.num_examples // batch_size):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={
            x: batch_xs, y: batch_ys})
    print(sess.run(accuracy,
                   feed_dict={
                       x: mnist.validation.images,
                       y: mnist.validation.labels,
                       
                   }))

0.964
0.9776
0.9742
0.9802
0.9788
